# Snow Depth Estimates with ICESat-2

This notebook uses a combination of ICESat-2 and airborne lidar to derive snow depth. It uses data from the SnowEx 2023 campaign as an example, but can be applied to other locations if a shapefile or geoJSON is given.

This notebook is adapted from the 2023 ICESat-2 Hackweek, originally developed by Zachary Fair and Karina Zikan.

## User input

Acceptable field site IDs over Alaska are:
* `cffl`: Creamer's Field/Farmer's Loop
* `cpcrw`: Caribou/Poker Creek Experimental Watershed
* `bcef`: Bonanza Creek Experimental Forest
* `acp`: Arctic Coastal Plain
* `utk`: Toolik Research Station

Acceptable IDs for Sliderule ATL08 class (use numeric ID):
* `No classification:` -1
* `atl08_unclassified`: 0
* `atl08_noise`: 1
* `atl08_canopy`: 2
* `atl08_top_of_canopy`: 3
* `atl08_ground`: 4

In [1]:
# Field site ID
field_id = '/home/jovyan/shared-public/SnowPit/snow-depth/cffl_lidar_box.geojson'

# Snow-on (True) or snow-off (False) analysis
snow_on = True

# Use March UAF data ('mar') or October depths ('oct')
uaf_depths = 'mar'

# Base data path
path = '/home/jovyan/icesat2-snowex'

# Desired RGT and date range for data queries. Set rgt to "all" if
# all ground tracks are desired
date_range = ['2023-03-01', '2023-04-01']
rgt = '1356'

# SlideRule parameters (optional)
cnf_surface = 4
atl08_class = 4
segment_length = 40
res = 20

A breakdown of the SlideRule parameters above:

`cnf_surface`: The confidence level of the ICESat-2 photons.
* High-confidence photons (recommended for snow): 4
* High-/medium-confidence photons: 3
* High-/medium-/low-confidence photons: 2
* Signal photons (high/medium/low) and noise: 1
* Signal photons, noise, and solar background (not recommended): 0

`segment_length`: The along-track length to sample and aggregate photons, in meters. Currently set at 40 m, the resolution of the ATL06 product.

`res`: The along-track resolution of the returned data product. Currently set at 20 m to match ATL06.

## Read ICESat-2 data
To load the ICESat-2 data with minimal effort from the user, we will use SlideRule in the below function.

In [2]:
from sliderule import sliderule, icesat2

def atl06srq(field_geojson, date_range, rgt, cnf_surface, atl08_class, 
             segment_length, res):
    # Initiate SlideRule
    icesat2.init('slideruleearth.io', verbose=False)

    # Load geoJSON for site of interest
    region = sliderule.toregion(field_geojson)['poly']

    # Convert user-defined ATL08 class ID to string readable by SlideRule
    atl08_ids = {-1: 'None',
                 0: 'atl08_unclassified',
                 1: 'atl08_noise',
                 2: 'atl08_canopy',
                 3: 'atl08_top_of_canopy',
                 4: 'atl08_ground'}

    # Construct dictionary of parameters
    time_root = 'T00:00:00Z'
    parms = {
             "poly": region,
             "srt": icesat2.SRT_LAND,
             "cnf": cnf_surface,
             "len": segment_length,
             "res": res,
             "t0": date_range[0]+time_root,
             "t1": date_range[1]+time_root
            }

    # Check if all RGTs are considered, or only a subset
    if rgt != "all":
        parms["rgt"] = rgt
        print(f"Subsetting to only include ICESat-2 RGT {rgt}.")

    # Check for ATL08 filter
    if atl08_ids.get(atl08_class) != "None":
        parms["atl08_class"] = atl08_ids.get(atl08_class)
        print("Subsetting by selected ATL08 filter...")

    # Query SlideRule
    atl06sr = icesat2.atl06p(parms)

    return atl06sr

In [3]:
# Generate ICESat-2 data from SlideRule
atl06sr = atl06srq(field_id, date_range, rgt,
                 cnf_surface=cnf_surface,
                 atl08_class=atl08_class,
                 segment_length=segment_length,
                 res=res)

Subsetting to only include ICESat-2 RGT 1356.
Subsetting by selected ATL08 filter...


In [4]:
# Convert ATL06SR to geodataframe in EPSG:32606
atl06sr['lon'], atl06sr['lat'] = atl06sr.geometry.x, atl06sr.geometry.y
atl06sr_gdf = atl06sr.to_crs('epsg:32606')

In [5]:
atl06sr

,gt,pflags,rgt,region,dh_fit_dx,cycle,h_mean,n_fit_photons,h_sigma,spot,segment_id,rms_misfit,x_atc,y_atc,w_surface_window_final,geometry,lon,lat
time,,,,,,,,,,,,,,,,,,
2023-03-19 08:29:56.503992832,50,0,1356,5,-0.039522,18,173.646747,251,0.017774,5,640927,0.281565,12837520.0,-3208.230957,3.0,POINT (-147.66635 64.88849),-147.666349,64.888487
2023-03-19 08:29:56.506796032,50,0,1356,5,-0.028783,18,172.937818,214,0.017530,5,640928,0.256160,12837540.0,-3208.252930,3.0,POINT (-147.66639 64.88831),-147.666395,64.888309
2023-03-19 08:29:56.509600256,50,0,1356,5,-0.007885,18,172.593577,225,0.012808,5,640929,0.191549,12837560.0,-3208.287354,3.0,POINT (-147.66644 64.88813),-147.666441,64.888130
2023-03-19 08:29:56.512403200,50,0,1356,5,-0.016506,18,172.344564,250,0.014396,5,640930,0.227622,12837580.0,-3208.318604,3.0,POINT (-147.66649 64.88795),-147.666488,64.887952
2023-03-19 08:29:56.515205888,50,0,1356,5,-0.002490,18,172.124278,273,0.018082,5,640931,0.297664,12837600.0,-3208.341797,3.0,POINT (-147.66653 64.88777),-147.666534,64.887773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-19 08:29:57.310336000,60,0,1356,5,0.007658,18,147.042876,56,0.028571,6,641090,0.212940,12840783.0,-3298.644043,3.0,POINT (-147.67571 64.85948),-147.675714,64.859484
2023-03-19 08:29:57.313169152,60,0,1356,5,0.003375,18,147.159856,50,0.036398,6,641091,0.253199,12840803.0,-3298.643311,3.0,POINT (-147.67576 64.85931),-147.675760,64.859305
2023-03-19 08:29:57.316003584,60,0,1356,5,-0.004613,18,147.164322,52,0.039569,6,641092,0.278684,12840823.0,-3298.634521,3.0,POINT (-147.67581 64.85913),-147.675806,64.859127


## Read Airborne Lidar Data

To derive snow depth with ICESat-2, we need a snow-off digital elevation model (DEM), which commonly originates from airborne lidar. This next step is designed to load and prepare some airborne lidar data from the University of Alaska, Fairbanks for this analysis.

In [6]:
import earthaccess
import xarray as xr
earthaccess.login(strategy='interactive', persist=True)
auth = earthaccess.login()

In [7]:
region = sliderule.toregion(field_id)['poly']
coords = [(point["lon"], point["lat"]) for point in region]

In [8]:
# Coordinates for SW/NE corners
lon_min = min([coord[0] for coord in coords])
lat_min = min([coord[1] for coord in coords])
lon_max = max([coord[0] for coord in coords])
lat_max = max([coord[1] for coord in coords])

# Data query for lidar snow depth over Fairbanks, AK
results = earthaccess.search_data(
    short_name='SNEX23_Lidar',
    bounding_box = (lon_min, lat_min, lon_max, lat_max),
    temporal = ('2023-03-10', '2023-03-15')
)

# Open the resulting GeoTiff into Xarray
files = earthaccess.open(results)
lidar_snow_on = xr.open_mfdataset(files[1], engine='rasterio', chunks = {})

QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# Data query for lidar snow-off elevation over Fairbanks, AK
results = earthaccess.search_data(
    short_name='SNEX23_Lidar',
    bounding_box = (lon_min, lat_min, lon_max, lat_max),
    temporal = ('2022-05-20', '2022-05-31')
)

# Open the resulting GeoTiff into Xarray
files = earthaccess.open(results)
lidar_snow_off = xr.open_mfdataset(files[1], engine='rasterio', chunks = {})

QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
import pyproj

# Suppose the dataset's CRS is EPSG:32618 (UTM Zone 18N) -- replace with your actual CRS!
crs = pyproj.CRS.from_epsg(32606)
transformer = pyproj.Transformer.from_crs("EPSG:4326", crs, always_xy=True)

# Transform bounding box to projected coordinates
x_min, y_min = transformer.transform(atl06sr.lon.min(), atl06sr.lat.min())
x_max, y_max = transformer.transform(atl06sr.lon.max(), atl06sr.lat.max())
print([x_min, y_min, x_max, y_max])

# Ensure axis order corresponds to your dataset
#x0, x1 = sorted([x_min, x_max])
#y0, y1 = sorted([y_min, y_max])

print([lidar_snow_off.x.min().values, lidar_snow_off.y.min().values, lidar_snow_off.x.max().values, lidar_snow_off.y.max().values])

# Subset using .sel with slice
subset = lidar_snow_off['band_data'].sel(x=slice(x_min, x_max), y=slice(y_max, y_min))

subset

[467959.27738420665, 7192885.975504697, 468446.7464144385, 7196192.904681352]
[array(464281.), array(7192839.), array(468626.5), array(7196227.5)]


<xarray.DataArray 'band_data' (band: 1, y: 6614, x: 975)> Size: 26MB
dask.array<getitem, shape=(1, 6614, 975), dtype=float32, chunksize=(1, 1, 975), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 8kB 4.68e+05 4.68e+05 ... 4.684e+05 4.684e+05
  * y            (y) float64 53kB 7.196e+06 7.196e+06 ... 7.193e+06 7.193e+06
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area

In [11]:
subset_3m = subset.coarsen(x=6, y=6, boundary='trim').mean()
subset_3m

<xarray.DataArray 'band_data' (band: 1, y: 1102, x: 162)> Size: 714kB
dask.array<mean_agg-aggregate, shape=(1, 1102, 162), dtype=float32, chunksize=(1, 1, 162), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 1kB 4.68e+05 4.68e+05 ... 4.684e+05 4.684e+05
  * y            (y) float64 9kB 7.196e+06 7.196e+06 ... 7.193e+06 7.193e+06
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area

## Co-Locate ICESat-2 and UAF Lidar

For this step, we will co-locate ICESat-2 and UAF so that we can directly compare the two datasets. The co-location will use a statistical method called "spline interpolation", and we will perform this co-location with both the snow-on and snow-off data.

The below function has the code needed to perform the co-location.

In [12]:
# Packages needed for the below functions
import geopandas as gpd
import numpy as np
import pandas as pd
from scipy.interpolate import RectBivariateSpline

def colocate_is2(lidar_snow_off, lidar_snow_on, is2_data):
    # Define x/y coordinates from snow-off data
    x0, y0 = np.array(lidar_snow_off.x), np.array(lidar_snow_off.y)

    # Do the same, but for the snow depth data
    xs, ys = np.array(lidar_snow_on.x), np.array(lidar_snow_on.y)

    # Remove filler values that would mess up the interpolator
    dem_heights = np.array(lidar_snow_off['band_data'].sel(band=1))[::-1,:]
    dem_heights[np.isnan(dem_heights)] = -9999
    dem_depths = np.array(lidar_snow_on['band_data'].sel(band=1))[::-1,:]
    dem_depths[np.isnan(dem_depths)] = -9999

    # Generate interpolator schemes
    interp_height = RectBivariateSpline(np.array(y0)[::-1], 
                                        np.array(x0),
                                        dem_heights)
    interp_depth = RectBivariateSpline(np.array(ys)[::-1],
                                       np.array(xs),
                                       dem_depths)

    # Use the spline interpolator to align the lidar with ICESat-2
    is2_lidar_df = pd.DataFrame()
    for beam in np.unique(is2_data['gt']):
        # Subset ICESat-2 data by current beam
        is2_tmp = is2_data.loc[is2_data['gt']==beam]

        # ICESat-2 x/y coordinates
        xn, yn = is2_tmp.geometry.x, is2_tmp.geometry.y

        # Define indices within x/y bounds of DEM
        i1 = (xn>np.min(x0)) & (xn<np.max(x0))
        i1 &= (yn>np.min(y0)) & (yn<np.max(y0))

        # Estimate lidar elevation and snow depth along ICESat-2 track
        lidar_height = interp_height(yn[i1], xn[i1], grid=False)
        lidar_depth = interp_depth(yn[i1], xn[i1], grid=False)

        # Construct dataframe of ICESat-2 and lidar data
        tmp = pd.DataFrame(data={'lat': is2_tmp['lat'][i1],
                                 'lon': is2_tmp['lon'][i1],
                                 'x': xn[i1],
                                 'y': yn[i1],
                                 'rgt': is2_tmp['rgt'][i1],
                                 'beam': is2_tmp['gt'][i1],
                                 'is2_height': is2_tmp['h_mean'][i1],
                                 'n_fit_photons': is2_tmp['n_fit_photons'][i1],
                                 'h_sigma': is2_tmp['h_sigma'][i1],
                                 'dh_fit_dx': is2_tmp['dh_fit_dx'][i1],
                                 'lidar_height': lidar_height,
                                 'lidar_snow_depth': lidar_depth
                                    }
                              )
        # Concatenate the co-located data into  final DataFrame
        is2_lidar_df = pd.concat([is2_lidar_df, tmp])

    return is2_lidar_df

In [ ]:
# Use the above function to co-locate the airborne lidar and ICESat-2
atl06sr_uaf = colocate_is2(lidar_snow_off, lidar_snow_on, atl06sr)

# Estimate the ICESat-2 snow depth
atl06sr_uaf['is2_snow_depth'] = atl06sr_uaf['is2_height'] - atl06sr_uaf['lidar_height']

# Convert final DataFrame in GeoDataFrame
atl06sr_uaf_gdf = gpd.GeoDataFrame(atl06sr_uaf,
                                   geometry=gpd.points_from_xy(atl06sr_uaf.lon, atl06sr_uaf.lat),
                                   crs='EPSG:4326')

atl06sr_uaf_gdf

An outline of the variables in our current GeoDataFrame:
* `lat` and `lon`: The latitude and longitude along the ICESat-2 track.
* `x` and `y`: The easting and northing along the ICESat-2 track, in projection EPSG:32606.
* `rgt`: The reference ground track number of the ICESat-2 track of interest.
* `beam`: The ICESat-2 beam designation (gt1l, gt2r, etc.)
* `is2_height`: ICESat-2 height estimate at the given location.
* `n_fit_photons`: Number of ICESat-2 photons used to derive `is2_height`.
* `h_sigma`: Approximate uncertainty of `is2_height`.
* `dh_fit_dx`: A rough measure of surface slope along the ICESat-2 track.
* `lidar_height`: Lidar height estimate at the given location.
* `lidar_snow_depth`: Lidar snow depth estimate at the given location.
* `is2_snow_depth`: ICESat-2 snow depth estimate at the given location.

The key variables are `is2_snow_depth` and `lidar_snow_depth` for our comparisons. Several of the other variables, such as `n_fit_photons` and `h_sigma`, can be used to filter or process the data further, if desired.

Let's look at a simple comparison between the two depth products.

In [ ]:
import matplotlib.pyplot as plt

# Line plot of along-track snow depths
fig, ax = plt.subplots(figsize=(9,6))
atl06sr_uaf_gdf.plot(kind='scatter', ax=ax, x='lat', y='is2_snow_depth',
                     linewidth=3, label='ICESat-2')
atl06sr_uaf_gdf.plot(kind='scatter', ax=ax, x='lat', y='lidar_snow_depth',
                     linewidth=1.5, label='UAF lidar')
ax.set_xlabel('Latitude', fontsize=14)
ax.set_ylabel('Snow depth [m]', fontsize=14)
ax.set_ylim([0, 1.5])
ax.legend()

We can also calculate the difference in snow depth between ICESat-2 and UAF, then make a spatial plot using `geopandas.explore()`.

In [ ]:
# Calculate snow depth bias
atl06sr_uaf_gdf['snow_depth_residual'] = atl06sr_uaf_gdf['is2_snow_depth'] - atl06sr_uaf_gdf['lidar_snow_depth']

# Create a spatial plot of the snow depth bias
atl06sr_uaf_gdf.explore(column='snow_depth_residual', 
                        tiles='Esri.WorldImagery',
                        cmap='viridis',
                        vmin=-1.5, vmax=1.5)

If the data looks good, then we can save the final GeoDataFrame as a geoJSON.

In [ ]:
# Save the GeoDataFrame
atl06sr_uaf_gdf.to_file(f'{path}/is2_uaf_snow-depths.geojson',
                        driver='GeoJSON')